In [1]:
# Import liberaries and functions
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import re

## Risk Disclosures

In [2]:
# Load RF data 
topics_df = pd.read_csv("Data/RDdf_BERT3.csv", parse_dates=['report_dt', 'filing_dt'])
print(topics_df.shape)
print(topics_df.columns)
print(topics_df['Topic_H'].nunique())

(1698148, 18)
Index(['CIK', 'report_dt', 'filing_dt', 'ticker', 'rf_seq', 'SIC', 'TopicWO',
       'Topic', 'Topic_H', 'rf_length', 'Pa', 'Pr', 'Fu', 'Sentiment', 'FOG',
       'FF', 'Specificity', 'SIC3'],
      dtype='object')
7169


In [3]:
# Specify covid 19 topics

# BERTopic_info = pd.read_csv("BERT/BERTopic_info.csv").drop(columns='Count')
# covid_topics = BERTopic_info[BERTopic_info["Representation"].str.contains(r"covid")]["Topic"].tolist()
# # BERTopic_info[BERTopic_info['Topic'].isin(covid_topics)].set_index("Topic", drop=True).to_records()
# covid_topics.remove(11627)
# topics_df['Covid_RF'] = topics_df['Topic'].isin(covid_topics).astype(int)

BERT_DTM = pd.read_csv("BERT/BERTopic_DTM3.csv")
covid_topics = BERT_DTM[BERT_DTM["Words"].str.contains(r"covid")][['Topic', 'Timestamp', 'Frequency']]

topics_df['Timestamp'] = topics_df['report_dt'].dt.year

topics_df['Covid_RF'] = pd.merge(
    left=topics_df,
    right=covid_topics,
    on=['Topic', 'Timestamp'],
    how='left'
)['Frequency'].notna().astype(int)

topics_df.drop(columns='Timestamp', inplace=True)

In [4]:
# if the RF is forward-looking
topics_df['Fu_RF'] = (topics_df['Fu']/topics_df[['Pa', 'Pr', 'Fu']].sum(axis=1) > topics_df['Pa']/topics_df[['Pa', 'Pr', 'Fu']].sum(axis=1))

# if the RF is backward-looking
topics_df['Pa_RF'] = (topics_df['Fu']/topics_df[['Pa', 'Pr', 'Fu']].sum(axis=1) < topics_df['Pa']/topics_df[['Pa', 'Pr', 'Fu']].sum(axis=1))

In [5]:
# Aggregate the records at the CIK-year level
agg_tops = (
    topics_df.groupby(["CIK", "report_dt", "filing_dt", "FF"])[
        ['SIC3', 'ticker', 'rf_length', 'Specificity', 'Pa_RF', 'Fu_RF', 'Sentiment', 'FOG', 'Covid_RF']
    ]
    .agg({
        'rf_length': 'sum',
        'SIC3' : 'unique',
        'ticker' : 'unique',
        'Specificity': 'sum',
        'Pa_RF': 'sum', 'Fu_RF': 'sum',
        'Sentiment': 'mean', 
        'FOG': 'mean',
        'Covid_RF': 'sum'
    }).reset_index()
).drop_duplicates(subset=["CIK", "filing_dt", "report_dt"]).sort_values(["CIK", "filing_dt", "report_dt"])

agg_tops['length_1'] = agg_tops.drop_duplicates(subset=['CIK', 'filing_dt']).groupby(["CIK"])['rf_length'].shift(1)
agg_tops['length_1'] = agg_tops.groupby('CIK')['length_1'].ffill()
agg_tops['Delta_length'] = agg_tops['rf_length'] - agg_tops['length_1']

agg_tops["ticker"] = agg_tops["ticker"].map(lambda x: x[0])
agg_tops["SIC3"] = agg_tops["SIC3"].map(lambda x: x[0])

In [6]:
# Determine generic and non-generic topics
topics_df['Topic_freq'] = topics_df.groupby(['Topic_H'])['rf_seq'].transform('count')
topics_df['generic'] = (topics_df['Topic_freq']>topics_df["Topic_H"].value_counts().quantile(0.5)).astype(int)

df = topics_df[topics_df["generic"]==1].groupby(["CIK", "report_dt", "filing_dt"])['Topic_H']\
        .nunique().rename("Generics").reset_index()

agg_tops = pd.merge(
    left=agg_tops,
    right=df,
    on=["CIK", "filing_dt", "report_dt"],
    how='left'
)

In [7]:
# Risk factor clusters disclosed and not disclosed per report 
disc_df = pd.pivot_table(
    topics_df, index = ["CIK", "filing_dt", "report_dt"], 
    columns='Topic_H', values='rf_seq'
).notna().astype(int).sort_values(["CIK", "filing_dt", "report_dt"]).reset_index()

disc_df['ryear'] = disc_df["report_dt"].dt.year

# Drop firm-year observations with more than 1 report in one fiscal year
disc_df.drop_duplicates(subset=disc_df.columns.difference(["filing_dt", "report_dt"]), keep="first", inplace=True)

In [8]:
N = topics_df['Topic_H'].max() +1
# Difference between disclosed risk topics in 2 consecutive years
disc_diff = disc_df.filter(range(N)) - disc_df.groupby("CIK")[disc_df.filter(range(N)).columns].shift(1)

# Number of added, repeated and removed individual topics
disc_df["reported"] = disc_df.filter(range(N)).sum(axis=1)
disc_df["added"] = disc_diff.dropna(how='all')[disc_diff>0].sum(axis=1)
disc_df["removed"] = disc_diff.dropna(how='all')[disc_diff<0].sum(axis=1)
disc_df["repeated"] = disc_df["reported"] - disc_df["added"]

In [9]:
# # duplicated ryears with report at the begining of the year
# disc_df['ryear_dupd'] = disc_df.duplicated(subset=['CIK', 'ryear'], keep='last')

# disc_df["ryear-1"] = disc_df.groupby('CIK')['ryear'].shift(1)

# # change ryear if duplicated and there is a gap between two report years 
# disc_df['ryear'] = disc_df[['ryear_dupd', 'ryear', 'ryear-1']].apply(
#     lambda x: x['ryear']-1 if x['ryear_dupd'] and x['ryear']-1>x['ryear-1'] else x['ryear'],
#     axis=1
# )

# disc_df = disc_df\
#     .drop_duplicates(subset=['CIK', 'ryear'], keep='first')\
#         .reset_index(drop=True).drop(columns=['ryear_dupd', 'ryear-1'])

In [10]:
df = disc_df[['CIK', 'filing_dt', 'report_dt', 
            'reported', 'repeated', 'added', 'removed']]

stat_data = pd.merge(
    left=agg_tops,
    right=df,
    on=['CIK', 'filing_dt', 'report_dt'],
    how='left'
)
# Number of days from fiscal year end and actual filing date
stat_data['rfGap'] = (stat_data['filing_dt'] - stat_data['report_dt']).dt.days

stat_data['fyear'] = stat_data['filing_dt'].dt.year
stat_data['ryear'] = stat_data['report_dt'].dt.year

## Other variables

In [11]:
# If firm appoints a chief risk officer (CRO)
CRO = pd.read_csv("Data/CRO.csv")
CRO.dropna(subset='title', inplace=True)
CRO['CRO'] = CRO['title'].str.contains(r"(chief risk)|(risk officer)", regex=True, case=False).astype(int)
CRO = CRO.groupby(['year', 'ticker'])['CRO'].max().reset_index()

C:\Users\u0147656\AppData\Local\Temp\ipykernel_18348\1931497266.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  CRO['CRO'] = CRO['title'].str.contains(r"(chief risk)|(risk officer)", regex=True, case=False).astype(int)


In [12]:
stat_data["CRO"] = pd.merge(
    left=stat_data,
    right=CRO,
    left_on=['ryear', 'ticker'],
    right_on=['year', 'ticker'],
    how='left'
)['CRO']

In [13]:
# Load BoardEX data
compo = pd.read_csv("Data/Board-Composition.csv", 
                    parse_dates=['AnnualReportDate'], 
                    usecols=['RoleName', 'AnnualReportDate', 'CIKCode', 'BoardID', 'DirectorID']).drop_duplicates()

committees = pd.read_csv(
    "Data/BoardEx_Committees.csv", parse_dates=['AnnualReportDate']
).drop_duplicates().reset_index(drop=True)

In [14]:
# Check CRO in BoardEx database
compo['Board_CRO'] = compo['RoleName'].str.contains(r"(chief risk)|(risk officer)", regex=True, case=False).astype(int)

# If director is in a risk committee
committees['RiskCommittee'] = committees['CommitteeName'].str.contains(r"risk", case=False).astype(int)

# Only directors that are in risk committee (some directors are in multiple committees - to remove duplicates)
Risk_committee = committees.loc[
    committees['RiskCommittee']==1,
    ['AnnualReportDate', 'RiskCommittee', 'BoardID', 'DirectorID']
].drop_duplicates().reset_index(drop=True)

compo = pd.merge(
    left=compo,
    right=Risk_committee,
    on=['AnnualReportDate', 'BoardID', 'DirectorID'],
    how='left'
).fillna({'RiskCommittee': 0})

compo['ryear'] = pd.to_datetime(compo['AnnualReportDate']).dt.year

compo_sum = compo.groupby(['CIKCode', 'ryear'])[['Board_CRO', 'RiskCommittee']].agg(
    {'Board_CRO': 'max', 'RiskCommittee': 'sum'}
).reset_index()

compo_sum[['CIKCode', 'ryear']] = compo_sum[['CIKCode', 'ryear']].astype(int)

C:\Users\u0147656\AppData\Local\Temp\ipykernel_18348\2998693217.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  compo['Board_CRO'] = compo['RoleName'].str.contains(r"(chief risk)|(risk officer)", regex=True, case=False).astype(int)


In [15]:
stat_data["Board_CRO"] = pd.merge(
    left=stat_data,
    right=compo_sum,
    left_on=['ryear', 'CIK'],
    right_on=['ryear', 'CIKCode'],
    how='left'
)['Board_CRO']

stat_data.fillna({"CRO": stat_data["Board_CRO"]}, inplace=True)

stat_data.drop(columns=['Board_CRO'], inplace=True)

stat_data["RC"] = pd.merge(
    left=stat_data,
    right=compo_sum,
    left_on=['ryear', 'CIK'],
    right_on=['ryear', 'CIKCode'],
    how='left'
)['RiskCommittee']

In [ ]:
other_data = pd.read_csv('Data/Study2_data1_V2.csv', parse_dates=['report_dt', 'filing_dt'])

In [17]:
# Different Tobin's Q calculation (Florio et al., 2017)
financials = pd.read_csv("Data\Financials3.csv", parse_dates=['datadate'])
financials['ryear'] = financials['datadate'].dt.year
financials['rmonth'] = financials['datadate'].dt.month
financials["TobinQ2"] = (financials["mkvalt"]+financials["lt"]) / financials["at"].replace(0, np.nan)
financials["ROA"] = financials["ni"] / financials["at"].replace(0, np.nan)
financials["ROA2"] = financials["ebit"] / financials["at"].replace(0, np.nan)

# Calculate industry ROA
financials.fillna({'sich': financials['sic']}, inplace=True)
financials['SIC3'] = financials['sich'].map(lambda x: f"{int(x):04d}"[:3] if ~np.isnan(x) else x)
financials['IndROA'] = financials.groupby(['SIC3', 'ryear'])['ROA'].transform('mean')

financials['cik'] = financials['cik'].astype(int)

# drop duplicated fiscal year/month
financials.sort_values(["cik", "ryear", "rmonth", 'TobinQ2'], inplace=True)
financials.drop_duplicates(subset=["cik", "ryear", "rmonth"], keep='first', inplace=True)

In [18]:
financials.columns

Index(['cik', 'datadate', 'act', 'at', 'dt', 'ebit', 'ebitda', 'intan', 'lct',
       'lt', 'ni', 'revt', 'seq', 'teq', 'xopr', 'xrd', 'xt', 'naicsh', 'sich',
       'mkvalt', 'naics', 'sic', 'ryear', 'rmonth', 'TobinQ2', 'ROA', 'ROA2',
       'SIC3', 'IndROA'],
      dtype='object')

In [19]:
stat_data.shape

(38343, 25)

In [20]:
Study3_data = pd.merge(
    left=stat_data,
    right=other_data[[
        'CIK', 'report_dt', 'filing_dt', 'COUNT_WEAK', 'Big4', 'GenderRatio', 'NumberDirectors', 'Age',
        'LnkdFirm', 'Independent', 'Volatility+30', 'Volatility_30', 'Volatility+60', 'Volatility_120', 
        'SHRTURN', 'Beta_126', 'NUMBEROFANALYSTS', 'DtA', 'ROE', 'NPM', 'mkvalt', 'logMC', 'at', 'logTA', 
        'INTtA', 'Current', 'TobinQ', 'BtM', 'RDxopr', 'ProprietaryCost', 'IndVol_', 'InstOwnership', 'ROA',
    ]],
    on=['CIK', 'filing_dt', 'report_dt'],
    how='left'
).drop(columns='ticker')

In [21]:
Study3_data["rmonth"] = Study3_data['report_dt'].dt.month

Study3_data = pd.merge(
    left=Study3_data,
    right=financials[['cik', 'ryear', 'rmonth', 'TobinQ2', 'IndROA', 'ROA2']],
    left_on=["CIK", "ryear", "rmonth"],
    right_on=["cik", "ryear", "rmonth"],
    how="left"
)

financials.sort_values(["cik", "ryear", 'TobinQ2'], inplace=True)
financials.drop_duplicates(subset=["cik", "ryear"], keep='first', inplace=True)

df = pd.merge(
    left=Study3_data,
    right=financials[['cik', 'ryear', 'TobinQ2', 'IndROA', 'ROA2']],
    left_on=["CIK", "ryear"],
    right_on=["cik", "ryear"],
    how="left",
    suffixes=['', '_2']
)

Study3_data.fillna({'TobinQ2': df['TobinQ2_2']}, inplace=True)
Study3_data.fillna({'IndROA': df['IndROA_2']}, inplace=True)
Study3_data.fillna({'ROA2': df['ROA2_2']}, inplace=True)

In [23]:
Study3_data.shape

(38111, 57)

In [24]:
Study3_data.to_csv("Data/Study3_V2.csv")

In [ ]:
Study3_data.columns

Index(['CIK', 'ryear', 'report_dt', 'filing_dt', 'FF', 'rf_length', 'SIC3',
       'Specificity', 'Pa_RF', 'Fu_RF', 'Sentiment', 'FOG', 'Covid_RF',
       'length_1', 'Delta_length', 'Generics', 'reported', 'repeated', 'added',
       'removed', 'rfGap', 'fyear', 'CRO', 'RC', 'COUNT_WEAK', 'Big4',
       'GenderRatio', 'NumberDirectors', 'Age', 'LnkdFirm', 'Independent',
       'Volatility+30', 'Volatility_30', 'Volatility+60', 'Volatility_120',
       'SHRTURN', 'Beta_126', 'NUMBEROFANALYSTS', 'DtA', 'ROE', 'NPM',
       'mkvalt', 'logMC', 'at', 'logTA', 'INTtA', 'Current', 'TobinQ', 'BtM',
       'RDxopr', 'ProprietaryCost', 'IndVol_', 'InstOwnership', 'ROA',
       'TobinQ2', 'IndROA'],
      dtype='object')